In [39]:
using PyPlot

using QuantumOptics
using LinearAlgebra
using Interpolations
using SplitApplyCombine
using DifferentialEquations
using SimpleDiffEq

#Random number generation
using Statistics
using Distributions
using Random

#Physical constants
using PhysicalConstants.CODATA2018: c_0, k_B, m_u
using Unitful

#Read and write CSV files
using DataFrames
using CSV

#Check time left to execute and benchmark functions
using ProgressMeter
using BenchmarkTools

In [40]:
include("../src/atom_sampler.jl");
include("../src/lasernoise_sampler.jl");
include("../src/utilities.jl");
include("../src/rydberg_model.jl");

$\hat{H} = -\Delta(t) \hat{n}_p - \delta(t) \hat{n}_r + \frac{\Omega_r(t)}{2} e^{i \phi_r(t)}\hat{\sigma}_{gp}+ \frac{\Omega_b(t)}{2}e^{i \phi_b(t)}\hat{\sigma}_{pr}+h.c.$

- $\phi_r(t), \phi_b(t)$ - phase noise from red and blue laser


- $\Omega_r(t), \Omega_b(t)$ - atom dynamics and laser amplitude noise


- $\Delta(t), \delta(t)$ - Doppler shifts for red and blue laser


- $\left| g \right>,\left| p \right>,\left| r \right>, \left| gt \right>$ - basis

__Params__

atom_params = [$m, T$]


trap_params = [$U_0, w_0, z_0$]

___

red_laser_params = [$ \Omega_0^{(r)}, w_0^{(r)}, z_0^{(r)}$]

blue_laser_params = [$ \Omega_0^{(b)}, w_0^{(b)}, z_0^{(b)}$]

___
red_laser_phase_params = [$h_0^{(r)}, h_g^{(r)}, \sigma_g ^{(r)}, f_g^{(r)}$]

blue_laser_phase_params = [$h_0^{(b)}, h_g^{(b)}, \sigma_g ^{(b)}, f_g^{(b)}$]
___

detuning_params = [$\Delta_0, \delta_0$]


decay_params = [$\Gamma_g, \Gamma_{gt}$]
___

contrast_params = [$\varepsilon, \varepsilon', \eta$]

___



__Simulation pipeline__

__1.__ Generate N samples of atom initial conditions in trap $(x_i, v_i, z_i, vx_i, vy_i, vz_i)$.

- input: atom_params, trap_params



- output: N samples of initial conditions $[[x_i, y_i, z_i, vx_i, vy_i, vz_i],...]$



__2.__ Generate phase amplitudes of blue and red laser: $S_\phi^{(r)}, S_\phi^{(b)}$

- input: red_laser_phase_params, blue_laser_phase_params



- output: amplitudes_red, amplitudes_blue



__3.__ Run simulation 


- input: t, samples, amplitudes_red, amplitudes_blue, detuning_params


- output: averaged density matrix $\rho(t) = \frac{1}{N}\overset{N}{\underset{i=1}{\sum }}\rho_i(t)$ for states $\left| g \right>, \left| p \right>, \left| r \right>, \left| g' \right>$




__4.__ Calculate expectation values

__Unit scales__

$E = \varepsilon E_0, \;\;\; T = t E_0 , \;\;\; E_0 = k_B \cdot 1\mu K$





$M = m m_u, \;\;\; m_u \simeq 1.66\cdot10^{-27} kg$


$V = v v_0, \;\;\; v_0 = 1 \mu m/ \mu s, \;\;\; vconst = \sqrt{\frac{E0}{m_u}} \simeq 0.09\mu m /\mu s$


$R = r r_0, \;\;\; r_0 = 1\mu m$

In [42]:
c = ustrip(u"m/s", c_0);  #Speed of light
kB = ustrip(u"J/K", k_B)  #Boltzmann constant
mu = ustrip(u"kg", m_u);  #Unit of atomic mass


m = 86.9091835;       #Rb87 mass in a.u.
E0 = kB * 1e-6;       #Characteristic energy in μK
g0 = 9.81 * 1e-6;     #Gravity free fall acceleration
vconst = sqrt(E0/mu); #Useful constant for kinetic energy
r0 = 1e-6;            #Characteristic distance in m

_Trap frequencies_

$\omega_r = \sqrt{\frac{4 U_0}{m m_u w_0^2}} = \frac{2}{\sqrt{m}}\frac{1}{w_0} \sqrt{\frac{U_0}{E_0}}\sqrt{\frac{E_0}{m_u}} = 2 \frac{v_{const}}{w_0}  \sqrt{\frac{u_0}{m}}$,

$\omega_z = \sqrt{\frac{2 U_0}{m z_0^2}} = \sqrt{2} \frac{v_{const}}{w_0}  \sqrt{\frac{u_0}{m}}$.

___

_Atom dynamics_

$r(t)=r_i \cos(\omega t) + \frac{v_{i}}{\omega}\sin(\omega t)$,

$v(t)=v_{i}\cos(\omega t) - \omega r_i \sin(\omega t)$.
___

__Simulation__

In [10]:
function simulation(
        tspan, ψ0, 
        
        samples,
        
        f,
        red_laser_phase_amplitudes,
        blue_laser_phase_amplitudes,
        
        red_laser_params,
        blue_laser_params,
        
        detuning_params,
        decay_params;
        
        laser_noise=true,
        atom_dynamics=true
    )
    
    N = length(samples);
    ωr, ωz = trap_frequencies(atom_params, trap_params);
    
    Δ0, δ0 = detuning_params;
    J, Jdagger = JumpOperators(decay_params);
    
    ρ_mean = [zero(ψ0 ⊗ dagger(ψ0)) for _ ∈ 1:length(tspan)];
    
    @showprogress for i ∈ 1:N
        #Atom initial conditions
        xi, yi, zi, vxi, vyi, vzi = samples[i];
        

        #Atom trajectories
        X(t) = R(t, xi, vxi, ωr);
        Y(t) = R(t, yi, vyi, ωr);
        Z(t) = R(t, zi, vzi, ωz);
        Vz(t) = V(t, zi, vzi, ωz);


        #Generate phase noise traces for red and blue lasers
        ϕ_red_res = ϕ(tspan, f, red_laser_phase_amplitudes);
        ϕ_blue_res = ϕ(tspan, f, blue_laser_phase_amplitudes);

        #Interpolate phase noise traces to pass to hamiltonian
        nodes = (tspan, );
        ϕ_red = interpolate(nodes, ϕ_red_res, Gridded(Linear()));
        ϕ_blue = interpolate(nodes, ϕ_blue_res, Gridded(Linear()));
        
        #Hamiltonian params trajectories
        δ_temp(t) = δ(Vz(t), red_laser_params, blue_laser_params) .+ δ0;
        Δ_temp(t) = Δ(Vz(t), red_laser_params);
        Ωr_temp(t) = exp.(1.0im .* ϕ_red(t)) .* Ω(X(t), Y(t), Z(t), red_laser_params);
        Ωb_temp(t) = exp.(1.0im .* ϕ_blue(t)) .* Ω(X(t), Y(t), Z(t), blue_laser_params);
        
        #Hamiltonian
        H_temp(t) = TimeDependentSum(
        [
        t -> -Δ_temp(t),
        t -> -δ_temp(t),
        t -> Ωr_temp(t) ./2.0,
        t -> conj.(Ωr_temp(t)) ./2.0,
        t -> Ωb_temp(t)/2.0,
        t -> conj.(Ωb_temp(t)) ./2.0,
        ],
        
        [
        np,
        nr,
        σgp,
        σpg,
        σpr,
        σrp  
        ]
    );
        
        #Returns hamiltonian and jump operators in a form required by timeevolution.master_dynamic
#         super_operator(t, rho) = [H_temp(t), J, Jdagger];
        super_operator(t, psi) = H_temp(t);

        #Time evolution
#         tout, ρ = timeevolution.master_dynamic(tspan, ρ0, super_operator);
        
        tout, ψ = timeevolution.schroedinger_dynamic(tspan, ψ0, super_operator);
        ρ_mean = ρ_mean .+ ψ .⊗ dagger.(ψ);
    end;

    return ρ_mean/N
end;

__Stable version__

__Test__

In [12]:
λ = 0.852;
w0 = 1.2;
z0 = w0_to_z0(w0, λ)

5.309734062405284

In [13]:
atom_params = [m, 50.0];
trap_params = [1000.0, w0, z0];

samples, acc_rate = boltzmann_samples(trap_params, atom_params, 100; skip=5000, freq=1000);
acc_rate

0.2868952380952381

In [14]:
samples_visualise(samples)

In [15]:
#Saffman params
h0 = 13.0 * 1e-6;   #MHz^2/MHz
hg1 = 25.0 * 1e-6;  #MHz^2/MHz
hg2 = 2.0e3 * 1e-6; #MHz^2/MHz
fg1 = 130.0 * 1e-3; #MHz
fg2 = 234.0 * 1e-3; #MHz
σg1 = 18.0 * 1e-3;  #MHz
σg2 = 1.5 * 1e-3;   #MHz

red_laser_params = [2π * 14.0, 5.0, 5.0*3.68];
blue_laser_params = [2π * 14.0, 5.0, 5.0*3.68];

red_laser_phase_params  = [h0, [hg1, hg2], [σg1, σg2], [fg1, fg2]];
blue_laser_phase_params = [h0, [hg1, hg2], [σg1, σg2], [fg1, fg2]];

tspan = [0.0:0.01:15.0;];
f = [0.01:0.01:10.0;];

red_laser_phase_amplitudes = ϕ_amplitudes(f, red_laser_phase_params);
blue_laser_phase_amplitudes = ϕ_amplitudes(f, blue_laser_phase_params);

In [16]:
detuning_params = [2*π * 740.0, 0.0];
Γ = 2.0*π*6;
decay_params = [Γ/4, 3*Γ/4];

ψ0 = g;
ρ0 = ψ0⊗dagger(ψ0);

In [43]:
ρ_mean = simulation(
        tspan, ρ0, 
        
        samples,
        
        f,
        red_laser_phase_amplitudes,
        blue_laser_phase_amplitudes,
        
        red_laser_params,
        blue_laser_params,
        
        detuning_params,
        decay_params
    );

Progress:   2%|▉                                        |  ETA: 0:02:19

Progress:   3%|█▎                                       |  ETA: 0:01:48

Progress:   4%|█▋                                       |  ETA: 0:01:29

Progress:   5%|██                                       |  ETA: 0:01:19

Progress:   6%|██▌                                      |  ETA: 0:01:12

...

Progress:  96%|███████████████████████████████████████▍ |  ETA: 0:00:02

Progress:  97%|███████████████████████████████████████▊ |  ETA: 0:00:01

Progress:  98%|████████████████████████████████████████▏|  ETA: 0:00:01

Progress:  99%|████████████████████████████████████████▋|  ETA: 0:00:00

Progress: 100%|█████████████████████████████████████████| Time: 0:00:38


In [44]:
figure()
plot(tspan, expect(nr, ρ_mean));
ylim(0.0, 1.0)

maximum(real(expect(nr, ρ_mean)))
gcf()

MethodError: MethodError: no method matching expect(::Operator{NLevelBasis{Int64}, NLevelBasis{Int64}, Matrix{ComplexF64}}, ::Operator{CompositeBasis{Vector{Int64}, Tuple{NLevelBasis{Int64}, NLevelBasis{Int64}}}, CompositeBasis{Vector{Int64}, Tuple{NLevelBasis{Int64}, NLevelBasis{Int64}}}, Matrix{ComplexF64}})
Closest candidates are:
  expect(::Any, ::AbstractOperator{B1, B2}, !Matched::AbstractOperator{B3, B3}) where {B1, B2, B3<:CompositeBasis} at ~/.julia/packages/QuantumInterface/yvpXN/src/expect_variance.jl:6
  expect(::Any, ::AbstractOperator, !Matched::Vector) at ~/.julia/packages/QuantumInterface/yvpXN/src/expect_variance.jl:14
  expect(::Any, ::AbstractOperator{B, B}, !Matched::Ket{B2}) where {B, B2<:CompositeBasis} at ~/.julia/packages/QuantumOpticsBase/l5btd/src/operators.jl:114
  ...

ErrorException: syntax: incomplete: premature end of input

__Conclusion__

- It seems like $\Delta$ has very large numerical value (740 MHz), which makes problem stiff. I can switch ODE solver.

https://viralinstruction.com/posts/hardware/

__Ideas for improvement__

0. Julia has its own function profiler https://www.julia-vscode.org/docs/stable/userguide/profiler/ , https://docs.julialang.org/en/v1/stdlib/Profile/



1. Replace [t->Ω(t), t->Ω(t)], [σab, σba] with single operator.



2. Try to include detuning fluctuations into phase noise, less time-dependent terms in hamiltonian.



3. Check time it takes to run interpolation of phase noise and compare it w/o interpolation.



4. Maybe there are some recepies in Bloqade.jl how to speed up code.